In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
#stop_words=set(nltk.corpus.stopwords.words('english'))

In [2]:
df = pd.read_csv("abcnews-date-text.csv")
df.head(10)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
5,20030219,ambitious olsson wins triple jump
6,20030219,antic delighted with record breaking barca
7,20030219,aussie qualifier stosur wastes four memphis match
8,20030219,aust addresses un security council over iraq
9,20030219,australia is locked into war timetable opp


In [3]:
# drop the publish date.
df.drop(['publish_date'],axis=1,inplace=True)

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
import pandas as pd
import numpy as np
import re
import string

def text_preprocess(tweet):
  tweet = tweet.lower()
  tweet = re.sub('(\r\n|rt|#|amp)','',tweet)
  tweet = re.sub('0+[a-z0-9]*','',tweet)
  doc = nlp(tweet)
  cleaned_tokens = [token.lemma_ for token in doc if ((token.text not in  nlp.Defaults.stop_words) and (token.text not in string.punctuation) and (token.is_space == False))]
  cleaned_tweet = " ".join(token for token in cleaned_tokens)
  return cleaned_tweet

In [6]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
import string,re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [8]:

df['headline_cleaned_text'] = df['headline_text'].apply(text_preprocess)

In [9]:
df.head()

,headline_text,headline_cleaned_text
0,aba decides against community broadcasting lic...,aba decide community broadcasting licence
1,act fire witnesses must be aware of defamation,act fire witness aware defamation
2,a g calls for infrastructure protection summit,g call infrastructure protection summit
3,air nz staff in aust strike for pay rise,air nz staff aust strike pay rise
4,air nz strike to affect australian travellers,air nz strike affect australian traveller


In [10]:
df.drop(['headline_text'],axis=1,inplace=True)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
model = CountVectorizer(max_df = 0.95,min_df=2,max_features=1000,ngram_range=(1,2),stop_words='english')
model_vect = model.fit_transform(df['headline_cleaned_text'].values)
vocab = model.get_feature_names()

In [12]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,random_state=42,max_iter=1)

In [13]:
lda_top=lda_model.fit_transform(model_vect)

In [14]:
print(lda_top.shape)  # (no_of_doc,no_of_topics)
print(lda_top)

(426297, 5)
[[0.05134032 0.05062912 0.79659887 0.05107629 0.05035539]
 [0.06778517 0.06692448 0.06789909 0.73052434 0.06686692]
 [0.28251142 0.0507485  0.05091322 0.56501403 0.05081283]
 ...
 [0.05105174 0.79669942 0.05060591 0.05105554 0.0505874 ]
 [0.06725224 0.06776525 0.50140452 0.06866788 0.29491012]
 [0.58919004 0.1024443  0.10288586 0.10111878 0.10436103]]


In [15]:
print(lda_model.components_)
print(lda_model.components_.shape)

[[ 62.9613972  112.56515042 130.26183607 ... 270.18353991 143.05968891
   48.25627233]
 [111.6665269  188.41125177  18.71313384 ...  70.83646701 445.57221993
  217.48606112]
 [  3.63918254  86.36572944 192.43249871 ... 374.13835919 153.08214511
   60.25153488]
 [  0.70170372  79.42634486 156.16915374 ... 236.55242643  36.41164784
   92.95728671]
 [292.03118964 125.23152352 144.42337764 ... 173.28920746 117.87429821
   36.04884496]]
(5, 1000)


In [16]:
vocab = model.get_feature_names()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
police govt council say new plan crash man death nsw 

Topic 1: 
kill man govt charge qld open set accuse pm case 

Topic 2: 
plan govt urge win council police cou group continue charge 

Topic 3: 
new help miss water seek govt die consider man sydney 

Topic 4: 
hit man death year plan fund push drug new charge 



In [18]:
!pip install mglearn

     |████████████████████████████████| 540 kB 30.8 MB/s 
  Created wheel for mglearn: filename=mglearn-0.1.9-py2.py3-none-any.whl size=582638 sha256=2381b927cd2f5fe9c88786e55339b0e1161933810f5a9845891566a54c28425a
  Stored in directory: /root/.cache/pip/wheels/f1/17/e1/1720d6dcd70187b6b6c3750cb3508798f2b1d57c9d3214b08b
Successfully built mglearn


In [19]:
import mglearn as mg
sorting = np.argsort(lda_model.components_)[:,::-1]
features = np.array(model.get_feature_names())
mg.tools.print_topics(topics=range(5),feature_names= features,sorting = sorting,topics_per_chunk = 5,n_words = 25)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
police        kill          plan          new           hit           
govt          man           govt          help          man           
council       govt          urge          miss          death         
say           charge        win           water         year          
new           qld           council       seek          plan          
plan          open          police        govt          fund          
crash         set           cou           die           push          
man           accuse        group         consider      drug          
death         pm            continue      man           new           
nsw           case          charge        sydney        charge        
water         claim         farmer        home          rise          
australia     face          qld           suppo         health        
win   